<a href="https://colab.research.google.com/github/alexdriedger/SlayTheSpireFightPredictor/blob/master/STSFightPredictor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import urllib.request
import json
import numpy as np
import collections
import datetime
import os
import pprint
from tensorflow import keras
from tensorflow.keras.layers import Input, Dense, Dropout, Concatenate, Average, Embedding, Lambda
#from tensorflow.keras.layers.merge import concatenate
from tensorflow.keras.models import Model, Sequential
from sklearn.preprocessing import MaxAbsScaler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
import random
import re
import tensorflow as tf
import datetime, os

%load_ext tensorboard

/home/ubuntu/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/ubuntu/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/ubuntu/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/ubuntu/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: 

In [2]:
CARD_INFO = json.load(open('cards_2020_11_14.json', 'r'))
RELIC_INFO = json.load(open('relics_2020_11_14.json', 'r'))

In [3]:
cached_data = False

if cached_data:
    loaded_data = np.load('cached_comp_data.npz')
    X = loaded_data['X']
    Y = loaded_data['Y']

else:
    json_data = list()
    for root, dirs, files in os.walk('a20_act1_defect'):
        for fname in files:
            path = os.path.join(root, fname)
            json_data.extend(json.load(open(path)))
    pprint.pprint(json_data[0])


if cached_data is False:
  print(f'Data has {len(json_data)} samples.')


{'ascension': 20,
 'cards': ['Strike_B',
           'Strike_B',
           'Strike_B',
           'Strike_B',
           'Defend_B',
           'Defend_B',
           'Defend_B',
           'Defend_B',
           'Zap',
           'Dualcast',
           'AscendersBane',
           'HandOfGreed',
           'Turbo'],
 'character': 'DEFECT',
 'damage_taken': 0,
 'enemies': 'Cultist',
 'entering_hp': 64,
 'floor': 3,
 'max_hp': 71,
 'not_picked': ['Melter', 'FTL'],
 'picked': 'Leap',
 'potion_used': False,
 'relics': ['Cracked Core', 'FaceOfCleric']}
Data has 141925 samples.


# Pre-Process Data

Load the json training examples and process them into vectors that can be used in a neural network

In [21]:
cce = tf.keras.losses.CategoricalCrossentropy(
    from_logits=False,
    label_smoothing=0,
    reduction="auto",
    name="categorical_crossentropy",
)

y_true = [[0, 1, 0], [0, 0, 1]]
y_pred = [[0.05, 0.95, 0], [0.1, 0.8, 0.1]]
cce(y_true, y_pred).numpy()
#1.177

# only_1s([[1, -1, 0], [0, 1, -1]])
# array([[1, 0, 0],
#       [0, 1, 0]], dtype=int32)>
def only_1s(x):
    return tf.where(tf.equal(x, 1), tf.ones_like(x), tf.zeros_like(x))

#dists = tf.constant([[.3,   .3,    .4],   [.2,    .5,    .3]])
#masks = tf.constant([[True, True, False], [False, True, True]])
#mask_renormalize_dist(dists, masks)
#array([[0.5  , 0.5  , 0.   ],
#       [0.   , 0.625, 0.375]], dtype=float32)>
def mask_renormalize_dist(dist, mask):
    masked_dist = dist * tf.cast(mask, tf.float32)
    norms = tf.norm(masked_dist, ord=1, axis=1)
    inv_norms = tf.transpose([tf.divide(tf.ones_like(norms), norms)])
    return tf.multiply(masked_dist, inv_norms)

def masked_cce(y_true, y_pred):
    # find the valid locations, (y_pred == 1 or -1)
    # turn y_pred -> 0 at invalid locations
    # map -1 in y_true to 0.
    # compute cce 
    allowed_choices = tf.not_equal(y_true, 0)
    y_pred_renormalized = mask_renormalize_dist(y_pred, allowed_choices)
    
    y_true_unrestricted = only_1s(y_true)
    return cce(y_true_unrestricted, y_pred_renormalized)


assert(masked_cce([[-1, 1, -1, 0], [1, -1, -1, 0]], [[.1, .2, .1, .5], [.3, .1, 0, .6]]).numpy()
       == cce([    [0, 1, 0],      [1, 0, 0]],      [[.2, .4, .2],     [.75, .25, 0]]).numpy())

print(cce([[1,0,0,0]], [[.25, .25, .25, .25]]))

tf.Tensor(1.3862944, shape=(), dtype=float32)


In [5]:
def upgrade_card(c):
    return c + "+1"

In [6]:
# Categories for one hot encoder. Categories are in alphabetical order and is the order used by OneHotEncoder
ALL_CARDS = [c for c in CARD_INFO if CARD_INFO[c]['color'] in ['BLUE', 'COLORLESS', 'CURSE']] + ['Strike', 'Defend']
ALL_CARDS = ALL_CARDS + [upgrade_card(c) for c in ALL_CARDS]
ALL_RELICS = list(RELIC_INFO.keys())
ALL_ENCOUNTERS = ['2 Fungi Beasts', '2 Louse', '2 Orb Walkers', '2 Thieves', '3 Byrds', '3 Cultists', '3 Darklings', '3 Louse', '3 Sentries', '3 Shapes', '4 Byrds', '4 Shapes', 'Apologetic Slime', 'Automaton', 'Awakened One', 'Blue Slaver', 'Book of Stabbing', 'Centurion and Healer', 'Champ', 'Chosen', 'Chosen and Byrds', 'Collector', 'Colosseum Nobs', 'Colosseum Slavers', 'Cultist', 'Cultist and Chosen', 'Donu and Deca', 'Exordium Thugs', 'Exordium Wildlife', 'Flame Bruiser 1 Orb', 'Flame Bruiser 2 Orb', 'Giant Head', 'Gremlin Gang', 'Gremlin Leader', 'Gremlin Nob', 'Hexaghost', 'Jaw Worm', 'Jaw Worm Horde', 'Lagavulin', 'Lagavulin Event', 'Large Slime', 'Looter', 'Lots of Slimes', 'Masked Bandits', 'Maw', 'Mind Bloom Boss Battle', 'Mysterious Sphere', 'Nemesis', 'Orb Walker', 'Red Slaver', 'Reptomancer', 'Sentry and Sphere', 'Shell Parasite', 'Shelled Parasite and Fungi', 'Shield and Spear', 'Slaver and Parasite', 'Slavers', 'Slime Boss', 'Small Slimes', 'Snake Plant', 'Snecko', 'Snecko and Mystics', 'Sphere and 2 Shapes', 'Spheric Guardian', 'Spire Growth', 'The Eyes', 'The Guardian', 'The Heart', 'The Mushroom Lair', 'Time Eater', 'Transient', 'Writhing Mass']
ALL_CHARACTERS = ['DEFECT', 'IRONCLAD', 'THE_SILENT', 'WATCHER']
ALL_CARD_CHOICES = ALL_CARDS + ['Singing Bowl', 'SKIP']


def make_key(l):
    indexes = {}
    for index, item in enumerate(l):
        assert item not in indexes, (item, l)
        indexes[item] = index
    return indexes
    
ALL_CARDS_KEY = make_key(ALL_CARDS)
ALL_RELICS_KEY = make_key(ALL_RELICS)
ALL_CHARACTERS_KEY = make_key(ALL_CHARACTERS)
ALL_ENCOUNTERS_KEY = make_key(ALL_ENCOUNTERS)
ALL_CARD_CHOICES_KEY = make_key(ALL_CARD_CHOICES)

## Pre-Process with Loops

Iterate through each training example and turn each field in the json data into each part of the vector needed for training.

This will later be optomized into vectorized code.

In [7]:
def encode_list2(list_to_encode, encoder_key):
    ret = np.zeros(len(encoder_key))
    for item in list_to_encode:
        ret[encoder_key[item]] += 1
    return ret

def encode_single2(value, encoder_key):
    return encode_list2([value], encoder_key)


In [8]:
def generalize_strikes_and_defends(cards):
  """
  Modifies any character specific Strikes and Defends (eg. Strike_R) into
  general Strikes and Defends(Strike)
  """
  for i, s in enumerate(cards):
    if s.startswith('Strike_') or s.startswith('Defend_'):
      cards[i] = re.sub('_.', '', s)
  return cards

def encode_cards(cards):
  return encode_list2(generalize_strikes_and_defends(cards), ALL_CARDS_KEY)

def encode_relics(relics):
  return encode_list2(relics, ALL_RELICS_KEY)

def encode_encounter(encounter):
  return encode_single2(encounter, ALL_ENCOUNTERS_KEY)

def encode_character(character):
  return encode_single2(character, ALL_CHARACTERS_KEY)

def encode_card_choice(sample):
    not_picked = list(sample['not_picked'])
    if sample['picked'] != 'SKIP':
        not_picked.append('SKIP')
    return (encode_single2(sample['picked'], ALL_CARD_CHOICES_KEY) 
            - encode_list2(not_picked, ALL_CARD_CHOICES_KEY))

In [9]:
# print(json_data[0]['picked'])
# print(json_data[0]['not_picked'])
# x = encode_card_choice(json_data[0])
# print(np.nonzero(x))
# print(x[np.nonzero(x)])
# print(ALL_CARD_CHOICES_KEY['Leap'], ALL_CARD_CHOICES_KEY['Melter'], 
#       ALL_CARD_CHOICES_KEY['FTL'], ALL_CARD_CHOICES_KEY['SKIP'])


In [10]:

def encode_sample_with_loop(sample):
  """
  Encode a single sample into a 1D vector
  """
  cards = encode_cards(sample['cards'])
  relics = encode_relics(sample['relics'])
  #relics = encode_relics(['Burning Blood'])
  encounter = encode_encounter(sample['enemies'])
  num_and_bool_data = np.array([sample['max_hp'] / 100.0, sample['entering_hp'] / 100.0, sample['ascension'], int(sample['potion_used'] == 'true')])  
  return np.concatenate((cards, relics, encounter, num_and_bool_data))

In [11]:
def decode_encoded_sample(row):
    cards = []
    relics = []
    lac =  len(ALL_CARDS)
    lar = len(ALL_RELICS)
    for index, quantity in enumerate(row):
        if quantity == 0:
            continue
        if index < lac:
            cards.extend([ALL_CARDS[index]] * int(quantity))
        elif index < lac + lar:
            relics.extend([ALL_RELICS[index - lac]] * int(quantity))
    return cards, relics

# a = json_data[0]
# enc = encode_sample_with_loop(a)
# b = decode_encoded_sample(enc)
# pprint.pprint(a)
# pprint.pprint(b)

In [12]:
# Set to True to test embedding experiments
USE_EMBEDDING = False

def preprocess_with_loop(data, log=True, log_freq=1000):
  """
  Pre-processes the data one sample at a time using loops.
  X has shape: (num_samples, training_vec_len)
  Y has shape: (num_samples)
  """
  processed_samples = []
  y = []
  skipped_no_picks = 0
  skipped_downsample = 0

  if log:
    print(f'Processing {len(data)} samples')
  
  for i, sample in enumerate(data):

    if log and i % log_freq == 0:
      print(f'{((i / len(data)) * 100):.1f} % complete. => {i} of {len(data)}, '
            f'skip_no_pick {skipped_no_picks}, '
            f'skipped_downsample {skipped_downsample}')

    if 'not_picked' not in sample or len(sample['not_picked']) == 0:
        skipped_no_picks += 1
        continue
        
    if sample['picked'] == 'Singing Bowl':
        continue
        
    #if sample['picked'] == 'SKIP':
     #   skipped_downsample += 1
    #    continue
        
    if 'PrismaticShard' in sample['relics']:
        continue
        
    if sample['max_hp'] > 200:
        continue
    
    if USE_EMBEDDING:
      processed_samples.append(encode_sample_embedding_with_loop(sample))
    else:
      processed_samples.append(encode_sample_with_loop(sample))
    
    #y.append(sample['damage_taken'])
    y.append(encode_card_choice(sample))
  X = np.vstack(processed_samples)
  Y = np.array(y, dtype='float32')
  return X, Y

# Deep Learning

The fun of the project! 😃 Let's build and training our model

In [13]:
# !pip install -U keras-tuner
# import kerastuner as kt

## Scaling and Spliting Data

Spliting the data into training and test set and scaling the features and labels to be between 0 and 1

In [14]:
if cached_data is False:
  X, Y = preprocess_with_loop(json_data, log_freq=15000)


Processing 141925 samples
0.0 % complete. => 0 of 141925, skip_no_pick 0, skipped_downsample 0
10.6 % complete. => 15000 of 141925, skip_no_pick 1832, skipped_downsample 0
21.1 % complete. => 30000 of 141925, skip_no_pick 3658, skipped_downsample 0
31.7 % complete. => 45000 of 141925, skip_no_pick 5520, skipped_downsample 0
42.3 % complete. => 60000 of 141925, skip_no_pick 7371, skipped_downsample 0
52.8 % complete. => 75000 of 141925, skip_no_pick 9226, skipped_downsample 0
63.4 % complete. => 90000 of 141925, skip_no_pick 11061, skipped_downsample 0
74.0 % complete. => 105000 of 141925, skip_no_pick 12911, skipped_downsample 0
84.6 % complete. => 120000 of 141925, skip_no_pick 14766, skipped_downsample 0
95.1 % complete. => 135000 of 141925, skip_no_pick 16616, skipped_downsample 0


In [15]:
def scale_X(X_data):
  """
  Used with one hot encoded model
  """
  X_copy = np.copy(X_data)
  max_abs_scaler = MaxAbsScaler()
  X_maxabs = max_abs_scaler.fit_transform(X_copy)
  with open('input_scales.json', 'w') as out_file:
    json.dump(max_abs_scaler.scale_.tolist(), out_file)
  return X_maxabs

def scale_Y(Y_data):
  return Y_data

In [23]:
pick_histogram = collections.Counter()

for game_state in json_data:
    if 'picked' in game_state:
        pick_histogram[game_state['picked']] += 1
print(pick_histogram)
print()
common_labels_with_counts = collections.Counter(np.argmax(Y, axis=1))

#print([(ALL_CARD_CHOICES[l], l, c) for l, c in common_labels_with_counts])


Counter({'SKIP': 26952, 'Ball Lightning': 5151, 'Cold Snap': 4370, 'Coolheaded': 4357, 'Conserve Battery': 3399, 'Defragment': 2684, 'Glacier': 2531, 'Gash': 2468, 'Singing Bowl': 2144, 'Go for the Eyes': 2115, 'Self Repair': 2046, 'Hologram': 1975, 'Sweeping Beam': 1881, 'Capacitor': 1873, 'Echo Form': 1844, 'Leap': 1829, 'Electrodynamics': 1783, 'Chill': 1751, 'Loop': 1739, 'Streamline': 1701, 'Beam Cell': 1602, 'White Noise': 1484, 'Core Surge': 1473, 'Compile Driver': 1454, 'Biased Cognition': 1350, 'FTL': 1278, 'Barrage': 1248, 'Seek': 1240, 'Genetic Algorithm': 1210, 'Buffer': 1176, 'Creative AI': 1147, 'BootSequence': 1133, 'Doom and Gloom': 1120, 'Turbo': 1105, 'Sunder': 1102, 'Static Discharge': 1070, 'Reinforced Body': 1044, 'Steam': 1028, 'Melter': 998, 'Undo': 983, 'Consume': 944, 'Fission': 940, 'Redo': 925, 'Rebound': 890, 'Rainbow': 843, 'Darkness': 817, 'Chaos': 786, 'Lockon': 756, 'Auto Shields': 697, 'Machine Learning': 692, 'Reboot': 688, 'Storm': 685, 'All For One':

In [17]:
tf.keras.backend.clear_session()
model = tf.keras.models.Sequential([
  tf.keras.layers.Dense(50, input_shape=(X.shape[1],), activation='relu'),
  tf.keras.layers.Dropout(.2),
  tf.keras.layers.Dense(50, activation='relu'),
  tf.keras.layers.Dropout(.2),
  tf.keras.layers.Dense(len(ALL_CARD_CHOICES), activation='softmax')
])

model.summary()
#model.compile(
#    optimizer=keras.optimizers.RMSprop(learning_rate=.001),
#    loss='mean_absolute_error',
#    metrics=['mean_absolute_error', 'mean_squared_error'])
model.compile(
    loss=masked_cce,
    optimizer='adam',
    metrics=['accuracy']
   )

# Tensorboard
logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

#X_scaled = scale_X(X)
X_scaled = X # try this?
Y_scaled = scale_Y(Y)
X_train, X_test, Y_train, Y_test = train_test_split(
    X_scaled, Y_scaled, test_size=0.33, shuffle=False)

history = model.fit(X_train, Y_train, 
                    #sample_weight=weights_train,
                    batch_size=32, epochs=20, validation_split=0.2, callbacks=[tensorboard_callback])

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 50)                27450     
_________________________________________________________________
dropout (Dropout)            (None, 50)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 50)                2550      
_________________________________________________________________
dropout_1 (Dropout)          (None, 50)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 296)               15096     
Total params: 45,096
Trainable params: 45,096
Non-trainable params: 0
_________________________________________________________________
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Train on 65383 samples, 

InvalidArgumentError: Nan in summary histogram for: dense/kernel_0 [Op:WriteHistogramSummary] name: dense/kernel_0/

In [ ]:
history = model.fit(X_train, Y_train, batch_size=32, epochs=5, validation_split=0.2, callbacks=[tensorboard_callback])

In [ ]:
test_scores = model.evaluate(X_test, Y_test, verbose=2)

print("Test loss:", test_scores[0])
print("Test accuracy:", test_scores[1])

In [ ]:
out = model.predict(X_test)

In [ ]:
valid_options_out = np.multiply(out, np.absolute(Y_test))

In [ ]:
def cn(index):
    return ALL_CARD_CHOICES[index]

In [ ]:
ct = 0

for example, predictions, actuals in zip(X_test, out, Y_test):
    model_pick = np.argmax(np.multiply(predictions, np.absolute(actuals)))
    real_label = np.argmax(actuals)
    if model_pick == real_label: 
        print('correct')
    else: 
        print('wrong')
    choice_indexes =  np.nonzero(actuals)[0]
    choices = list(map(cn, choice_indexes))
    print(choices, cn(model_pick), cn(real_label))
    print(decode_encoded_sample(example))
    ct += 1
    print()
    if ct > 10:
        break

In [ ]:
c = collections.Counter(np.argmax(np.multiply(out, np.absolute(Y_test)), axis=1) )
c.most_common(10)


### Tensorboard

Inspect training and validation results with tenorboard and upload logs to tensorboard.dev

In [ ]:
%tensorboard --logdir logs

In [ ]:
# !tensorboard dev upload --logdir ./logs \
#   --name "Slay the Spire fight predictions" \
#   --description "Training results predicting health loss in a Slay the Spire fight"

### Manual Inspection

In [ ]:
def inspect_test_cases():
  """
  For manually looking at test predictions vs actual number
  """
  num_test = 50
  print(X_test.shape)
  pred = model.predict(X_test[:num_test])
  pred_actual = Y_test[:num_test]
  pred_actual = pred_actual[..., np.newaxis]
  out = np.concatenate((pred, pred_actual), axis=1)
  np.set_printoptions(precision=3, suppress=True)
  print(out)
inspect_test_cases()

## Tune the Model

Let's try to find the best hyper parameters

In [ ]:
def build_model(hp):
  model = tf.keras.models.Sequential()
  for i in range(hp.Int('num_layers_big', 1, 2)):
    model.add(tf.keras.layers.Dense(
        units=hp.Int('units_' + str(i), min_value=100, max_value=500, step=50, default=400), activation='relu'))
    tf.keras.layers.Dropout(
      hp.Float('dropout', 0.1, 0.4, step=0.1, default=0.2))
  for i in range(hp.Int('num_layers_small', 1, 2)):
    model.add(tf.keras.layers.Dense(
        units=hp.Int('units_' + str(i), min_value=16, max_value=128, step=16, default=32), activation='relu'))
    tf.keras.layers.Dropout(
      hp.Float('dropout', 0, 0.3, step=0.1, default=0.2))
  model.add(tf.keras.layers.Dense(1))
  model.compile(
      optimizer=keras.optimizers.RMSprop(hp.Choice('learning_rate', [1e-1, 1e-2, 1e-3, 1e-4])),
      loss='mean_absolute_error',
      metrics=['mean_absolute_error', 'mean_squared_error'])
  return model

def tune_model():
  tuner = kt.Hyperband(
      build_model,
      objective='val_loss',
      max_epochs=20,
      hyperband_iterations=4)
  tuner.search(X_train, Y_train,
              epochs=5,
              validation_data=(X_test, Y_test))
  return tuner

# tuner = tune_model()
# tuner.results_summary()

## Save the Model

Save the model to be loaded into the game to be used in a mod!

Cache the data to help speed up development

In [ ]:
model.save("STSFightPredictor") # Saved Model Format
# model.save("STSFP.h5")

np.savez_compressed('cached_comp_data.npz', X=X, Y=Y)

In [ ]:
# For testing in the mod that the model in the mod is predicting the save values
for i in range(3):
  case = X_test[i]
  sb = ''
  sb = 'float[] testCase = {'
  for num in case:
    sb += str(num)
    sb += '0f, '
  sb += '};'
  print(sb)

## Embedding Experiments

The current model has a good training loss curve but the validation curve looks more like a straight line than a curve. This problem is often related to overfit. Rather than one hot encoding cards, relics, and encounter, an experiment with Embedding layers was run to try to reduce overfit.

An Embedding layer can be used to learn relations between cards. The general idea is to encode cards, relics, and enemies as vectors instead of a single numbers.

Because there are a variable number of cards and relics a player can have, the average of card vectors and the average of relic vectors was taken. These averages can be passed into a Dense layer.

In [ ]:
# Custom average function to ignore masked layers
def avg_labmda_fun(x, mask):
  mask_cast = keras.backend.cast(mask, 'float32')
  expanded = keras.backend.expand_dims(mask_cast)
  count = tf.keras.backend.sum(mask_cast)
  sum = keras.backend.sum(expanded * x, axis=1)
  return sum / count

In [ ]:
def train_embedding_model():
  # Embed cards and average output vectors
  card_input = Input(shape=(NUM_CARDS_FOR_EMB, ), name='cards_input')
  card_embedding = Embedding(len(ALL_CARDS) + 1, 26, mask_zero=True)(card_input)
  card_average = Lambda(avg_labmda_fun, output_shape=(26, ), mask=None)(card_embedding)

  # Embed relics and average output vectors
  relic_input = Input(shape=(NUM_RELICS_FOR_EMB, ), name='relics_input')
  relic_embedding = Embedding(len(ALL_RELICS) + 1, 13, mask_zero=True)(relic_input)
  relic_average = Lambda(avg_labmda_fun, output_shape=(13, ), mask=None)(relic_embedding)

  # Embed encounter. There is only a single encounter but the lambda is used to reshape the vector
  encounter_input = Input(shape=(1, ), name='encounter_input')
  encounter_embedding = Embedding(len(ALL_ENCOUNTERS), 8)(encounter_input)
  encounter_layer_reshape = Lambda(lambda x: keras.backend.mean(x, axis=1), output_shape=(8, ))(encounter_embedding)

  numbers_input = Input(shape=(4, ), name='num_and_bool_input')

  # Concatenate before sending to Dense layers
  merged = concatenate([card_average, relic_average, encounter_layer_reshape, numbers_input])

  dense_1 = Dense(40, activation='relu')(merged)
  drop_out_1 = Dropout(.1)(dense_1)
  dense_out = Dense(1)(drop_out_1)


  emb_model = Model(inputs=[card_input, relic_input, encounter_input, numbers_input], output=dense_out)
  emb_model.summary()

  emb_model.compile(
      optimizer=keras.optimizers.RMSprop(learning_rate=.0001),
      loss='mse',
      metrics=['mae'])

  logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
  tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

  Y_scaled = scale_Y(Y)
  X_train, X_test, Y_train, Y_test = train_test_split(X, Y_scaled, test_size=0.33, shuffle=False)

  cards_col = X_train[:, 0:NUM_CARDS_FOR_EMB]
  relic_index = NUM_CARDS_FOR_EMB + NUM_RELICS_FOR_EMB
  relics_col = X_train[:, NUM_CARDS_FOR_EMB:relic_index]
  encounter_index = relic_index + 1
  encounter_col = X_train[:, relic_index:encounter_index]
  num_and_bool_col = X_train[:, encounter_index:]
  max_abs_scaler = MaxAbsScaler()
  num_and_bool_col = max_abs_scaler.fit_transform(num_and_bool_col)

  history = emb_model.fit(x={'cards_input': cards_col, 'relics_input': relics_col, 'encounter_input': encounter_col, 'num_and_bool_input': num_and_bool_col}, y=Y_train, batch_size=32, epochs=20, validation_split=0.2)
  return emb_model

if USE_EMBEDDING:
  emb_model = train_embedding_model()

In [ ]:
def inspect_embeddings():
  embeddings = emb_model.layers[3].get_weights()[0]
  embeddings.shape
  weights = dict()
  for i, name in enumerate(ALL_CARDS):
    weights[name] = embeddings[i]
    # print(f'{name}:\t{embeddings[i]}')

  print(weights['Pommel Strike'])
  print(weights['Sucker Punch'])

if USE_EMBEDDING:
  inspect_embeddings()